In [1]:
#https://ipython.readthedocs.io/en/stable/interactive/magics.html
get_ipython().run_line_magic('config', 'IPCompleter.greedy = True')

In [14]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math as mt
from sklearn.preprocessing import Imputer


In [15]:
#==============================================================================
# Importing the dataset
#==============================================================================
dataset_new = pd.read_csv('big_case_train.csv')

dataset_test = pd.read_csv('puzzle_train_dataset.csv')


In [16]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [ ]:
#=============================================================================
#Reorganize data
#==============================================================================
dataset = pd.read_csv('puzzle_train_dataset.csv')
imputer_mode = Imputer(missing_values="NaN",strategy="most_frequent",axis=0)
dataset.iloc[:,[1]]=imputer_mode.fit_transform(dataset.iloc[:,[1]])
sz = len(dataset)
defaulters = range(0,sz)
for i in defaulters:
    if dataset['default'][i] == True:
        dataset = dataset.append(dataset.iloc[i,:],ignore_index=True)
for i in log_progress(defaulters):
     if dataset['default'][i] == True:
        dataset.drop(i,inplace=True)
#Save data to csv for reuse
dataset.to_csv('reorganized.csv', sep=',', index=False)
 

In [ ]:
#selectingh the reorganized csv file to save time
#Use Celll Reorganize data to get this file.
dataset = pd.read_csv('reorganized.csv')

dataset_test = pd.read_csv('puzzle_test_dataset.csv')

In [ ]:
#==============================================================================
#Calculating the total profit
#==============================================================================
profit = dataset_new['spends']*0.05 + dataset_new['revolving_balance']*0.17 - dataset_new['card_request']*10 - dataset_new['minutes_cs']*2.6

#Calculating the absolute profit after considering the inflation costs
dataset_new['absolute_profit'] = profit - profit * dataset_new['month']*0.005 

#Adding the processed data from big_data.csv to our training dataset.
#dataset_new['default'] = dataset['default']

In [ ]:
#==============================================================================
#creating np array to calculate total profits after considering revolving balance
#==============================================================================
big_data = dataset_new.iloc[:,[0,-1,4]].values
ids = np.ndarray(shape=(np.unique(big_data[:,0]).size,4),dtype=float)
ids[:,0] = dataset['default']
big_data